In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import cftime
import cartopy.crs as ccrs
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
from matplotlib.colors import BoundaryNorm
import matplotlib.cm as cm

# Fish

In [ ]:
fish = pd.read_csv('..data/Big_DF.csv')

cols = ['Year',"LondonHerring001", "MunichHerring001old",
        'GdanskHerring001','DutchHerring001.old',
        "A1Cod003",
        'MunichStockfish001old',
        'ViennaStockfish001',
        'DutchStockFish001.old','LondonSaltedCod001',
        "FeSpanCod001","AntwerpHerring001","ViennaHerring001", "A1SS001"
]

fish = fish[cols]

fish_long = pd.melt(fish, id_vars=['Year'], var_name='LocationSpecies', value_name='Price')

fish_long = fish_long.dropna()

#count number of years in each location
print(fish_long.groupby('LocationSpecies')['Year'].count())

#rename locations
fish_long['LocationSpecies'] = fish_long['LocationSpecies'].replace({'LondonHerring001': 'LondonHerring',
                                                           'GdanskHerring001': 'GdanskHerring',
                                                           'DutchHerring001.old': 'AmsterdamHerring',
                                                           'FeSpanCod001': 'BarcelonaCod',
                                                           'AntwerpHerring001': 'AntwerpHerring',
                                                           'ViennaHerring001': 'ViennaHerring',
                                                        'MunichHerring001old': 'MunichHerring',
                                                        'DutchStockFish001.old': 'AmsterdamCod',
                                                                'MunichStockfish001old': 'MunichCod',
                                                                'A1SS001': 'ParisHerring',
                                                                'ViennaStockfish001': 'ViennaCod',
                                                                'A1Cod003': 'ParisCod',
                                                                'LondonSaltedCod001': 'LondonCod'})
fish_long[['Location', 'Species']] = fish_long['LocationSpecies'].str.extract(r'([A-Za-z]+)(Herring|Cod)')



LocationSpecies
A1Cod003                  36
A1SS001                   42
AntwerpHerring001        215
DutchHerring001.old      190
DutchStockFish001.old    139
FeSpanCod001             168
GdanskHerring001         211
LondonHerring001         295
LondonSaltedCod001        86
MunichHerring001old       70
MunichStockfish001old     74
ViennaHerring001         119
ViennaStockfish001        63
Name: Year, dtype: int64


In [3]:
fish_long

,Year,LocationSpecies,Price,Location,Species
0,1259,LondonHerring,0.057062,London,Herring
5,1264,LondonHerring,0.091840,London,Herring
7,1266,LondonHerring,0.064828,London,Herring
8,1267,LondonHerring,0.077659,London,Herring
9,1268,LondonHerring,0.069218,London,Herring
...,...,...,...,...,...
8283,1670,ParisHerring,1080.000000,Paris,Herring
8313,1700,ParisHerring,1200.000000,Paris,Herring
8319,1706,ParisHerring,960.000000,Paris,Herring
8360,1747,ParisHerring,1800.000000,Paris,Herring


In [4]:
from geopy.geocoders import Nominatim
import time

cities = fish_long.Location.unique()

geolocator = Nominatim(user_agent="city_geocoder")

city_to_coords = {}

for city in cities:
    try:
        location = geolocator.geocode(city)
        if location:
            city_to_coords[city] = (location.latitude, location.longitude)
            print(f"{city}: ({location.latitude}, {location.longitude})")
        else:
            print(f"{city}: Not found")
    except Exception as e:
        print(f"Error with {city}: {e}")
    time.sleep(1)  # To avoid getting blocked by the server

fish_long['Latitude'] = fish_long['Location'].map(lambda x: city_to_coords[x][0] if x in city_to_coords else None)
fish_long['Longitude'] = fish_long['Location'].map(lambda x: city_to_coords[x][1] if x in city_to_coords else None)

London: (51.5074456, -0.1277653)
Munich: (48.1371079, 11.5753822)
Gdansk: (54.4288032, 18.798327)
Amsterdam: (52.3730796, 4.8924534)
Paris: (48.8534951, 2.3483915)
Vienna: (48.2083537, 16.3725042)
Barcelona: (41.3825802, 2.177073)
Antwerp: (51.2211097, 4.3997081)


# PDSI

In [ ]:
# Load OWDA and compute weighted PDSI
owda = xr.open_dataset("..data/owda.nc")
owda["PDSI_weighted"] = np.sqrt(np.cos(np.deg2rad(owda["lat"])) + 1e-6) * owda["pdsi"]

# Flatten the 3D (time, lat, lon) data into a long table of gridpoints
pdsi_grid = owda["PDSI_weighted"].stack(points=("lat", "lon")).reset_coords()

# Drop points where all values are NaN
valid_points = pdsi_grid.dropna(dim="points", how="all")

# Extract coordinates for KDTree
grid_coords = np.column_stack([valid_points["lat"].values, valid_points["lon"].values])
tree = cKDTree(grid_coords)

# Prepare city coordinates
city_coords = np.array(list(city_to_coords.values()))  # shape (n_cities, 2)

# Find nearest grid point for each city
distances, indices = tree.query(city_coords, k=1, workers = -1)

# For each city, extract the time series from the nearest grid point
selected_data = []
years = owda["time"].values

for i, (city, coord) in enumerate(city_to_coords.items()):
    grid_idx = indices[i]
    lat = valid_points["lat"].values[grid_idx]
    lon = valid_points["lon"].values[grid_idx]

    # Get the PDSI series from that grid point
    city_ts = owda["PDSI_weighted"].sel(lat=lat, lon=lon, method="nearest")
    city_ts = city_ts.assign_coords(Location=city)
    selected_data.append(city_ts)

# Combine the city time series (DataArrays) into a Dataset
pdsi_selected = xr.Dataset({
    'PDSI': xr.concat(selected_data, dim="loc_id")
})

# Rename time to Year
pdsi_selected = pdsi_selected.rename({"time": "Year"})

# Add domain averages as DataArrays (with the same time/Year dimension)
pdsi_selected["PDSI_europe"] = owda["PDSI_weighted"].sel(lat=slice(35, 70), lon=slice(-10, 40)).mean(dim=["lat", "lon"]).rename({"time": "Year"})
pdsi_selected["PDSI_southern"] = owda["PDSI_weighted"].sel(lat=slice(20, 40), lon=slice(-10, 20)).mean(dim=["lat", "lon"]).rename({"time": "Year"})

pdsi_df = pdsi_selected.to_dataframe().reset_index()
pdsi_df.rename(columns={"PDSI_weighted": "PDSI"}, inplace=True)

# Temp

In [ ]:
# Load dataset
xds = xr.open_dataset('..data/ModE-RA_ensmean_temp2_anom_wrt_1901-2000_1421-2008_mon.nc')

# Extract year and filter by given range
xds = xds.assign(Year=xds.time.dt.year)
xds = xds.sel(time=slice(str(1500), str(1850)))

# Apply latitude weighting
xds['temp'] = np.sqrt(np.cos(np.deg2rad(xds['latitude'])) + 1e-6) * (xds['temp2'])

# Define seasons
winter_months = [12, 1, 2]
summer_months = [4, 5, 6, 7]

def shift_decembers(time_array):
    return [t if t.month != 12 else cftime.DatetimeGregorian(t.year - 1, t.month, t.day) for t in time_array]

xds = xds.assign_coords(time=("time", shift_decembers(xds.time.values)))



# --- **(1) Compute City-Level Temperatures** ---
def extract_location_data(xds_season):
    return [
        xds_season.sel(latitude=lat, longitude=lon, method='nearest')
        .to_dataframe().reset_index().assign(Location=city)
        for city, (lat, lon) in city_to_coords.items()
    ]

# Compute city-level seasonal means
xds_winter_cities = xds.sel(time=xds.time.dt.month.isin(winter_months)).groupby('Year').mean(dim='time')
xds_summer_cities = xds.sel(time=xds.time.dt.month.isin(summer_months)).groupby('Year').mean(dim='time')

temp_winter = pd.concat(extract_location_data(xds_winter_cities), ignore_index=True)
temp_summer = pd.concat(extract_location_data(xds_summer_cities), ignore_index=True)

# --- **(2) Compute European Domain-Average Temperatures** ---
# Define European bounding box (adjust as needed)
lat_bounds = (70, 35)   # Latitude range for Europe
lon_bounds = (-15, 35)  # Longitude range for Europe

# Subset dataset to Europe only
xds_europe = xds.sel(latitude=slice(*lat_bounds), longitude=slice(*lon_bounds))

# Function to compute spatially averaged seasonal mean
def compute_seasonal_mean(xds, months):
    return (xds.sel(time=xds.time.dt.month.isin(months))
            .groupby('Year').mean(dim='time')
            .mean(dim=['latitude', 'longitude']))

# Compute European domain-average temperatures
xds_winter_europe = compute_seasonal_mean(xds_europe, winter_months)
xds_summer_europe = compute_seasonal_mean(xds_europe, summer_months)

# Convert to DataFrame
europe_temp_df = pd.DataFrame({
    'Year': xds_winter_europe.Year.values,
    'temp_winter_europe': xds_winter_europe['temp'].values,
    'temp_summer_europe': xds_summer_europe['temp'].values
})

# --- **(3) Merge Everything Together** ---
temp_df = pd.merge(temp_winter, temp_summer, on=['Year', 'Location'], suffixes=('_winter', '_summer'))

# Drop unnecessary columns
temp_df = temp_df.drop(columns=['latitude_winter', 'longitude_winter', 'latitude_summer', 'longitude_summer', 'temp2_winter', 'temp2_summer'])

# Merge with European averages
temp_df = pd.merge(temp_df, europe_temp_df, on='Year', how='left')




C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xarray\coding\times.py:1001: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xarray\core\indexing.py:516: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(self.get_duck_array(), dtype=dtype)


# Precip

In [ ]:
# Load dataset
xds_p = xr.open_dataset('..data/ModE-RA_ensmean_totprec_anom_wrt_1901-2000_1421-2008_mon.nc')

# Extract year and filter by given range
xds_p = xds_p.assign(Year=xds_p.time.dt.year)
xds_p = xds_p.sel(time=slice(str(1500), str(1850)))

# Apply latitude weighting
xds_p['precip'] = np.sqrt(np.cos(np.deg2rad(xds_p['latitude'])) + 1e-6) * xds_p['totprec']*86400*30

# Define seasons
winter_months = [12, 1, 2]
summer_months = [4, 5, 6, 7]

def shift_decembers(time_array):
    return [t if t.month != 12 else cftime.DatetimeGregorian(t.year - 1, t.month, t.day) for t in time_array]

xds_p = xds_p.assign_coords(time=("time", shift_decembers(xds_p.time.values)))



# --- **(1) Compute City-Level preciperatures** ---
def extract_location_data(xds_p_season):
    return [
        xds_p_season.sel(latitude=lat, longitude=lon, method='nearest')
        .to_dataframe().reset_index().assign(Location=city)
        for city, (lat, lon) in city_to_coords.items()
    ]

# Compute city-level seasonal means
xds_p_winter_cities = xds_p.sel(time=xds_p.time.dt.month.isin(winter_months)).groupby('Year').mean(dim='time')
xds_p_summer_cities = xds_p.sel(time=xds_p.time.dt.month.isin(summer_months)).groupby('Year').mean(dim='time')

precip_winter = pd.concat(extract_location_data(xds_p_winter_cities), ignore_index=True)
precip_summer = pd.concat(extract_location_data(xds_p_summer_cities), ignore_index=True)

# --- **(2) Compute European Domain-Average preciperatures** ---
# Define European bounding box (adjust as needed)
lat_bounds = (70, 35)   # Latitude range for Europe
lon_bounds = (-15, 35)  # Longitude range for Europe

# Subset dataset to Europe only
xds_p_europe = xds_p.sel(latitude=slice(*lat_bounds), longitude=slice(*lon_bounds))

# Function to compute spatially averaged seasonal mean
def compute_seasonal_mean(xds_p, months):
    return (xds_p.sel(time=xds_p.time.dt.month.isin(months))
            .groupby('Year').mean(dim='time')
            .mean(dim=['latitude', 'longitude']))

# Compute European domain-average preciperatures
xds_p_winter_europe = compute_seasonal_mean(xds_p_europe, winter_months)
xds_p_summer_europe = compute_seasonal_mean(xds_p_europe, summer_months)

# Convert to DataFrame
europe_precip_df = pd.DataFrame({
    'Year': xds_p_winter_europe.Year.values,
    'precip_winter_europe': xds_p_winter_europe['precip'].values,
    'precip_summer_europe': xds_p_summer_europe['precip'].values
})

# --- **(3) Merge Everything Together** ---
precip_df = pd.merge(precip_winter, precip_summer, on=['Year', 'Location'], suffixes=('_winter', '_summer'))

# Drop unnecessary columns
precip_df = precip_df.drop(columns=['latitude_winter', 'longitude_winter', 'latitude_summer', 'longitude_summer', 'totprec_winter', 'totprec_summer'])

# Merge with European averages
precip_df = pd.merge(precip_df, europe_precip_df, on='Year', how='left')


C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xarray\coding\times.py:1001: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xarray\core\indexing.py:516: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(self.get_duck_array(), dtype=dtype)


# ENSO

In [ ]:
enso3 = pd.read_csv("..data/cook2024-R15-ENSO-Rec-1500-2000.txt", delimiter="\t", comment="#", na_values="NA")
lat_lon = pd.read_csv("..data/cook2024-ENSO-latlon.txt", delimiter="\t", comment="#", na_values="NA")

# Melt the ENSO DataFrame to long format
enso_melted = enso3.melt(id_vars=["Year"], var_name="gridpoint", value_name="enso")
enso_melted["gridpoint"] = enso_melted["gridpoint"].astype(int)

# Merge with the lat/lon DataFrame
enso_merged = enso_melted.merge(lat_lon, on="gridpoint")



# Convert to xarray
enso_xr = enso_merged.set_index(["Year", "lat", "lon"])["enso"].to_xarray()

enso_xr = enso_xr  - enso_xr.sel(Year=slice(1801, 1900)).mean(dim="Year")

nino3 = enso_xr.sel(lat=slice(-5, 5), lon=slice(-150, -90)).mean(dim=["lat", "lon"])
nino34 = enso_xr.sel(lat=slice(-5, 5), lon=slice(-170, -120)).mean(dim=["lat", "lon"])
nino4 = enso_xr.sel(lat=slice(-5, 5), lon = slice(-200, -150)).mean(dim=["lat", "lon"])
nino12 = enso_xr.sel(lat=slice(-10, 0), lon=slice(-90, -80)).mean(dim=["lat", "lon"])

enso = pd.merge(nino3.to_dataframe().reset_index().rename(columns={"enso": "nino3"}), nino34.to_dataframe().reset_index().rename(columns={"enso": "nino34"}), on="Year")
enso = pd.merge(enso, nino4.to_dataframe().reset_index().rename(columns={"enso": "nino4"}), on="Year")
enso = pd.merge(enso, nino12.to_dataframe().reset_index().rename(columns={"enso": "nino12"}), on="Year")


enso.corr()

,Year,nino3,nino34,nino4,nino12
Year,1.000000,0.052022,0.046736,0.057840,0.060952
nino3,0.052022,1.000000,0.997527,0.992063,0.985244
nino34,0.046736,0.997527,1.000000,0.996980,0.973788
nino4,0.057840,0.992063,0.996980,1.000000,0.969020
nino12,0.060952,0.985244,0.973788,0.969020,1.000000


# JSL and NAO

In [ ]:
jsl = pd.read_excel("..data/reconstructed EU JSL.xlsx")

nao_cal = pd.read_excel("..data/nao_reconst.xlsx", sheet_name="Figure 2a", header=3)
nao_cal = nao_cal.rename(columns={"Time (years AD)" : "Year"})
nao_model = pd.read_excel("..data/nao_reconst.xlsx", sheet_name="Figure 2b", header=3)
nao_model = nao_model.rename(columns={"Time (years AD)" : "Year"})

nao = pd.merge(nao_cal, nao_model, on="Year", suffixes=('_cal', '_model'))

nao = nao.rename(columns={"Ensemble Mean_cal" : "NAO_cal", "Ensemble Mean_model" : "NAO_model"})[["Year", "NAO_cal", "NAO_model"]]

C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# Conflict

In [ ]:
# Load the data
post1400 = pd.read_excel('..data/Conflict-Catalog-18-vars.xlsx')
pre1400 = pd.read_excel('..data/Brecke-Pre-1400-European-Conflicts.xlsx')


# Convert StartYear and EndYear to integer if not NaN
post1400['StartYear'] = post1400['StartYear'].apply(lambda x: int(x) if not pd.isna(x) else np.nan)
post1400['EndYear'] = post1400['EndYear'].apply(lambda x: int(x) if not pd.isna(x) else np.nan)

# Filter Region to be 3 and 4
post1400 = post1400[post1400['Region'].isin([3, 4])]
# Select relevant columns
post1400 = post1400[['TotalFatalities', 'MilFatalities', 'StartYear', 'EndYear']].copy()
pre1400 = pre1400[['Fatalities', 'StartYear', 'EndYear']].copy()

# Fill NaN values in TotalFatalities and MilFatalities with 0
post1400['TotalFatalities'] = post1400['TotalFatalities'].fillna(0)
post1400['MilFatalities'] = post1400['MilFatalities'].fillna(0)
post1400['Deaths'] = post1400['TotalFatalities']

pre1400['Deaths'] = pre1400['Fatalities'].fillna(0)

# Drop TotalFatalities and MilFatalities
post1400 = post1400.drop(columns=['TotalFatalities', 'MilFatalities'])

pre1400 = pre1400.drop(columns=['Fatalities'])

wars = pd.concat([pre1400,post1400])

# Initialize an empty dictionary for tracking deaths per year
death_index = {}

# Populate the index with death counts, ongoing wars, started wars, and total duration
for _, row in wars.iterrows():
    if pd.isna(row['StartYear']) or pd.isna(row['EndYear']):
        continue
    
    year_range = range(int(row['StartYear']), int(row['EndYear']) + 1)
    deaths_per_year = row['Deaths'] / len(year_range)
    duration = len(year_range)
    
    for year in year_range:
        if year in death_index:
            death_index[year]['Deaths'] += deaths_per_year
            death_index[year]['ongoing_wars'] += 1
            if year == row['StartYear']:
                death_index[year]['started_wars'] += 1
                death_index[year]['total_duration'] += duration
        else:
            death_index[year] = {
                'Deaths': deaths_per_year,
                'ongoing_wars': 1,
                'started_wars': 1 if year == row['StartYear'] else 0,
                'total_duration': duration if year == row['StartYear'] else 0
            }

# Convert dictionary to DataFrame
conflict = pd.DataFrame.from_dict(death_index, orient='index').reset_index()

conflict['Death_ratio'] = conflict['Deaths'] / conflict['ongoing_wars']
conflict.rename(columns={'index': 'Year'}, inplace=True)
conflict = conflict.sort_values(by='Year').reset_index(drop=True).fillna(0)


C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# Merge

In [11]:
df_total = pd.merge(temp_df, pdsi_df, on=['Year', 'Location']) 
df_total = pd.merge(df_total, precip_df, on=['Year', 'Location'])
df_total = pd.merge(df_total, enso, on=['Year'])
df_total = pd.merge(df_total, jsl, on=['Year'])
df_total = pd.merge(df_total, nao, on=['Year'])
df_total = pd.merge(df_total, fish_long, on=['Year', 'Location'])
df_total = df_total.sort_values(['Location', 'Year'])

df_total = pd.merge(df_total, conflict, on=['Year'], how='left')
df_total[['Deaths', 'ongoing_wars', 'started_wars', 'total_duration', 'Death_ratio']] = df_total[['Deaths', 'ongoing_wars', 'started_wars', 'total_duration', 'Death_ratio']].fillna(0)
df_total['logprice'] = np.log(df_total['Price'])
df_total['logprice'] = df_total['logprice'].replace([np.inf, -np.inf], np.nan)

C:\Users\emile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [12]:
df_total

,Year,temp_winter,Location,temp_summer,temp_winter_europe,temp_summer_europe,loc_id,lon,lat,PDSI,...,Price,Species,Latitude,Longitude,Deaths,ongoing_wars,started_wars,total_duration,Death_ratio,logprice
0,1500,-0.929491,Amsterdam,0.046393,-0.583429,0.095119,3,4.75,52.25,-0.492157,...,42.360000,Herring,52.373080,4.892453,28533.333333,9.0,3.0,8.0,3170.370370,3.746205
1,1501,0.719225,Amsterdam,-0.066485,0.424553,-0.055762,3,4.75,52.25,-0.993704,...,36.000000,Herring,52.373080,4.892453,29233.333333,10.0,4.0,28.0,2923.333333,3.583519
2,1502,-0.373168,Amsterdam,-0.032026,-0.229643,-0.005292,3,4.75,52.25,-2.892695,...,32.400000,Herring,52.373080,4.892453,32066.666667,11.0,2.0,4.0,2915.151515,3.478158
3,1503,-0.661077,Amsterdam,0.171724,-0.387555,-0.038728,3,4.75,52.25,-4.137563,...,35.160000,Herring,52.373080,4.892453,28733.333333,10.0,2.0,9.0,2873.333333,3.559909
4,1504,0.109087,Amsterdam,0.340324,-0.077320,0.016197,3,4.75,52.25,-1.463952,...,48.000000,Herring,52.373080,4.892453,4533.333333,7.0,0.0,0.0,647.619048,3.871201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,1746,-0.116493,Vienna,-0.513565,0.542585,-0.293171,5,16.25,48.25,-2.052281,...,2.626875,Cod,48.208354,16.372504,37946.666667,2.0,0.0,0.0,18973.333333,0.965795
1447,1747,0.928594,Vienna,-0.631374,0.132982,-0.240880,5,16.25,48.25,-1.770755,...,2.626875,Cod,48.208354,16.372504,36696.666667,2.0,1.0,1.0,18348.333333,0.965795
1448,1748,-0.406176,Vienna,-0.115584,-0.030507,-0.149325,5,16.25,48.25,-2.405616,...,2.626875,Cod,48.208354,16.372504,36696.666667,1.0,0.0,0.0,36696.666667,0.965795
1449,1749,0.719155,Vienna,-0.197063,0.282024,-0.204976,5,16.25,48.25,-0.328039,...,2.560125,Cod,48.208354,16.372504,0.000000,0.0,0.0,0.0,0.000000,0.940056


In [13]:
df_total.corr(numeric_only=True)['logprice']

Year                    0.316467
temp_winter            -0.075540
temp_summer            -0.057072
temp_winter_europe     -0.005349
temp_summer_europe      0.010697
loc_id                  0.221851
lon                    -0.479551
lat                    -0.361388
PDSI                   -0.006685
PDSI_europe            -0.002700
PDSI_southern           0.038120
precip_winter          -0.133401
precip_summer          -0.123504
precip_winter_europe   -0.078178
precip_summer_europe   -0.055411
nino3                  -0.004704
nino34                 -0.005134
nino4                  -0.006188
nino12                 -0.004979
JSL                     0.034787
NAO_cal                 0.050045
NAO_model              -0.058912
Price                   0.530560
Latitude               -0.350096
Longitude              -0.480678
Deaths                  0.055272
ongoing_wars           -0.145276
started_wars           -0.094650
total_duration         -0.058127
Death_ratio             0.115175
logprice  

In [ ]:
df_total.to_csv("..processed data/fishprice_enso.csv", index=False)